# Браки и разводы.
Проект является экспериментальной лабораторной работой.

## Цели.
1. Чтение и анализ xml файлов ЕМИСС.
2. Визуализация результаов анализа с помощью Altair.
3. Сторителлинг на основе визуального анализа.
4. Генерация html контента на основе сторителлинга.

## Чтение и анализ xml файлов ЕМИСС.

In [15]:
# Загружаем библиотеки
import pandas as pd

In [16]:
# Создаём ссылку на xml файл
#data_raw='Число зарегистрированных браков (оперативные данные).xml'
data_raw='Число зарегистрированных разводов в расчете на 1000 населения (оперативные данные).xml'

### Что внутри xml.

Для чтения используется функция read_xml. Посмотрев xml файл можно увидеть, что есть несколько элементов, на которые можно сослаться:
1. Коды регионов.
2. Информация о переменной.
3. Год.
4. Значения переменной.

#### Коды регионов.

In [17]:
# Получение данных кодов регионов

namespaces_codes = 'http://www.SDMX.org/resources/SDMXML/schemas/v1_0/structure' # Имя объекта, где хранятся коды регионов

regines_codes = pd.read_xml(data_raw, # Ссылка на файл
                            xpath="//structure:Code", # Путь внутри файла до данных
                            namespaces = {"structure": namespaces_codes}) # Имя объекта
regines_codes # Результат

,value,Description
0,643,Российская Федерация
1,30,Центральный федеральный округ
2,14000000000,Белгородская область
3,15000000000,Брянская область
4,17000000000,Владимирская область
...,...,...
104,99000000000,Еврейская автономная область
105,77000000000,Чукотский автономный округ
106,39,Крымский федеральный округ
107,46001000000,Московская обл. в старых границах


In [18]:
regines_codes = regines_codes.rename(columns= {'value':'s_OKATO'}) # Переименовал колонку для будущего соединения таблиц
regines_codes['s_OKATO'] = regines_codes['s_OKATO'].astype(str) # Сделал значения в колонке текстом. Нужно будет для объединения таблиц
regines_codes

,s_OKATO,Description
0,643,Российская Федерация
1,30,Центральный федеральный округ
2,14000000000,Белгородская область
3,15000000000,Брянская область
4,17000000000,Владимирская область
...,...,...
104,99000000000,Еврейская автономная область
105,77000000000,Чукотский автономный округ
106,39,Крымский федеральный округ
107,46001000000,Московская обл. в старых границах


#### Информация о переменной.

In [19]:
namespaces_values = 'http://www.SDMX.org/resources/SDMXML/schemas/v1_0/generic'

regiones_values = pd.read_xml(data_raw, xpath="//generic:Value", namespaces = {"generic": namespaces_values})
regiones_values

,concept,value
0,s_OKATO,643
1,EI,"промилле (0,1 процента)"
2,PERIOD,январь
3,s_OKATO,643
4,EI,"промилле (0,1 процента)"
...,...,...
71152,EI,"промилле (0,1 процента)"
71153,PERIOD,декабрь
71154,s_OKATO,45001000000
71155,EI,"промилле (0,1 процента)"


Полученные данные идут подряд в определённой последовательности. Нужно первый столбец сделать названием колонок

In [20]:
regiones_values_pivot = regiones_values.pivot(columns = 'concept') # Делаю сводную таблицу из первого столбца
regiones_values_pivot

value                             
concept                       EI          PERIOD      s_OKATO
0                            NaN             NaN          643
1        промилле (0,1 процента)             NaN          NaN
2                            NaN          январь          NaN
3                            NaN             NaN          643
4        промилле (0,1 процента)             NaN          NaN
...                          ...             ...          ...
71152    промилле (0,1 процента)             NaN          NaN
71153                        NaN         декабрь          NaN
71154                        NaN             NaN  45001000000
71155    промилле (0,1 процента)             NaN          NaN
71156                        NaN  январь-декабрь          NaN

[71157 rows x 3 columns]

In [21]:
idx = pd.IndexSlice # Используем для того, чтобы забирать столбец из фрейма с мультииндексом

Я ещё толком не разобрался как работать с мультииндексом, но инфа тут
https://pandas.pydata.org/docs/user_guide/advanced.html

Далее нам нужно забрать отдельный столбец и удалить из него все пустые ячейки

In [22]:
PERIOD = regiones_values_pivot.loc[:, idx[:,'PERIOD']].dropna().reset_index(drop=True).T.reset_index(drop=True).T.rename(columns = {0:'PERIOD'})
PERIOD

,PERIOD
0,январь
1,январь-февраль
2,январь-март
3,январь-апрель
4,январь-май
...,...
23714,январь-октябрь
23715,ноябрь
23716,январь-ноябрь
23717,декабрь


Повторяем процедуру дважды. В будущем будет функция и цикл.

In [23]:
s_OKATO = regiones_values_pivot.loc[:, idx[:,'s_OKATO']].dropna().reset_index(drop=True).T.reset_index(drop=True).T.rename(columns = {0:'s_OKATO'})
s_OKATO

,s_OKATO
0,643
1,643
2,643
3,643
4,643
...,...
23714,45001000000
23715,45001000000
23716,45001000000
23717,45001000000


#### Год

In [25]:
regiones_Obs = pd.read_xml(data_raw, xpath="//generic:Obs", namespaces = {"generic": "http://www.SDMX.org/resources/SDMXML/schemas/v1_0/generic"})
regiones_Obs

,Time,ObsValue
0,2006,NaN
1,2006,NaN
2,2006,NaN
3,2006,NaN
4,2006,NaN
...,...,...
23714,2013,NaN
23715,2013,NaN
23716,2013,NaN
23717,2013,NaN


#### Значения

In [24]:
regiones_ObsValue = pd.read_xml(data_raw, xpath="//generic:ObsValue", namespaces = {"generic": "http://www.SDMX.org/resources/SDMXML/schemas/v1_0/generic"})
regiones_ObsValue

,value
0,"3,41"
1,"3,77"
2,"4,17"
3,"4,24"
4,"4,34"
...,...
23714,0
23715,0
23716,0
23717,0


## Сведение

In [34]:
concat = pd.concat([PERIOD,s_OKATO,regiones_Obs['Time'],regiones_ObsValue],axis=1)
concat

,PERIOD,s_OKATO,Time,value
0,январь,643,2006,"3,41"
1,январь-февраль,643,2006,"3,77"
2,январь-март,643,2006,"4,17"
3,январь-апрель,643,2006,"4,24"
4,январь-май,643,2006,"4,34"
...,...,...,...,...
23714,январь-октябрь,45001000000,2013,0
23715,ноябрь,45001000000,2013,0
23716,январь-ноябрь,45001000000,2013,0
23717,декабрь,45001000000,2013,0


## Консолидация

In [28]:
merge = concat.merge(regines_codes, on='s_OKATO')
merge

,PERIOD,s_OKATO,Time,value,Description
0,январь,643,2006,"3,41",Российская Федерация
1,январь-февраль,643,2006,"3,77",Российская Федерация
2,январь-март,643,2006,"4,17",Российская Федерация
3,январь-апрель,643,2006,"4,24",Российская Федерация
4,январь-май,643,2006,"4,34",Российская Федерация
...,...,...,...,...,...
20135,январь-октябрь,45001000000,2013,0,Москва в старых границах
20136,ноябрь,45001000000,2013,0,Москва в старых границах
20137,январь-ноябрь,45001000000,2013,0,Москва в старых границах
20138,декабрь,45001000000,2013,0,Москва в старых границах


In [35]:
merge['Source'] = data_raw
merge

,PERIOD,s_OKATO,Time,value,Description,Source
0,январь,643,2006,"3,41",Российская Федерация,Число зарегистрированных разводов в расчете на...
1,январь-февраль,643,2006,"3,77",Российская Федерация,Число зарегистрированных разводов в расчете на...
2,январь-март,643,2006,"4,17",Российская Федерация,Число зарегистрированных разводов в расчете на...
3,январь-апрель,643,2006,"4,24",Российская Федерация,Число зарегистрированных разводов в расчете на...
4,январь-май,643,2006,"4,34",Российская Федерация,Число зарегистрированных разводов в расчете на...
...,...,...,...,...,...,...
20135,январь-октябрь,45001000000,2013,0,Москва в старых границах,Число зарегистрированных разводов в расчете на...
20136,ноябрь,45001000000,2013,0,Москва в старых границах,Число зарегистрированных разводов в расчете на...
20137,январь-ноябрь,45001000000,2013,0,Москва в старых границах,Число зарегистрированных разводов в расчете на...
20138,декабрь,45001000000,2013,0,Москва в старых границах,Число зарегистрированных разводов в расчете на...


In [41]:
merge[['DataName','B']]=merge['Source'].str.split('.',1,expand=True)
merge = merge.drop(['B'], axis = 1)
merge

,PERIOD,s_OKATO,Time,value,Description,Source,DataName
0,январь,643,2006,"3,41",Российская Федерация,Число зарегистрированных разводов в расчете на...,Число зарегистрированных разводов в расчете на...
1,январь-февраль,643,2006,"3,77",Российская Федерация,Число зарегистрированных разводов в расчете на...,Число зарегистрированных разводов в расчете на...
2,январь-март,643,2006,"4,17",Российская Федерация,Число зарегистрированных разводов в расчете на...,Число зарегистрированных разводов в расчете на...
3,январь-апрель,643,2006,"4,24",Российская Федерация,Число зарегистрированных разводов в расчете на...,Число зарегистрированных разводов в расчете на...
4,январь-май,643,2006,"4,34",Российская Федерация,Число зарегистрированных разводов в расчете на...,Число зарегистрированных разводов в расчете на...
...,...,...,...,...,...,...,...
20135,январь-октябрь,45001000000,2013,0,Москва в старых границах,Число зарегистрированных разводов в расчете на...,Число зарегистрированных разводов в расчете на...
20136,ноябрь,45001000000,2013,0,Москва в старых границах,Число зарегистрированных разводов в расчете на...,Число зарегистрированных разводов в расчете на...
20137,январь-ноябрь,45001000000,2013,0,Москва в старых границах,Число зарегистрированных разводов в расчете на...,Число зарегистрированных разводов в расчете на...
20138,декабрь,45001000000,2013,0,Москва в старых границах,Число зарегистрированных разводов в расчете на...,Число зарегистрированных разводов в расчете на...
